In [189]:
import pandas as pd
import numpy as np
from datetime import timedelta
import statsmodels.api as sm
from os import listdir
from os.path import isfile, join



In [190]:
mypath = r'C:\Users\jjanko\Desktop\wei_mutual_fund_project\output\filter_cleaned'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
stata = pd.DataFrame()
error = []
for i in onlyfiles:
    if 'restricted' in i and 'legend' not in i:
        try:
            add = pd.read_csv(join(mypath, i),'rb', index_col = 0, delimiter=",", encoding='utf-8')
        except:
            error.append(i)
        if 'identifer' in add.columns:
            add['identifier'] = add['identifer'].copy()
            add.drop(labels = 'identifer', axis = 1, inplace = True)
        if stata.empty:
            stata = add.copy()
        else:
            stata = pd.concat([stata, add], axis = 0, sort = False , ignore_index=True)

In [191]:
stata

,holdings shares,holdings value,holdings name,key,fund name,identifier,value multiplier,textfile,CIK,date_filed,company conformed name,reporting_date,acq name,acq date,acq cost,restricted,weblink,Unnamed: 0.1
0,418804,232743,Foreign exchange contracts,(4),Strategic Multi-Asset Portfolio,Strategic Multi-Asset Portfolio,NaN,0000950123-11-100758.txt,726735.0,20111129.0,ANCHOR SERIES TRUST,20110930,NaN,NaN,NaN,yes,NaN,NaN
1,369192,3340855,"Dropbox, Inc.",(1)(2)(3)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-12-483973.txt,726735.0,20121128.0,ANCHOR SERIES TRUST,20120930,NaN,NaN,NaN,yes,NaN,NaN
2,369192,3371831,"Dropbox, Inc.",(1)(2)(3)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-13-240989.txt,726735.0,20130530.0,ANCHOR SERIES TRUST,20130331,NaN,NaN,NaN,yes,NaN,NaN
3,353044,3847591,"Dropbox, Inc.",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-13-454992.txt,726735.0,20131126.0,ANCHOR SERIES TRUST,20130930,NaN,NaN,NaN,yes,NaN,NaN
4,353044,6743564,"Dropbox, Inc.",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-14-216490.txt,726735.0,20140529.0,ANCHOR SERIES TRUST,20140331,NaN,NaN,NaN,yes,NaN,NaN
5,129662,2748056,"Cloudera, Inc.Series F",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-14-216490.txt,726735.0,20140529.0,ANCHOR SERIES TRUST,20140331,NaN,NaN,NaN,yes,NaN,NaN
6,291563,4495027,"One Kings Lane, Inc.",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-14-216490.txt,726735.0,20140529.0,ANCHOR SERIES TRUST,20140331,NaN,NaN,NaN,yes,NaN,NaN
7,8662,113753,"Docusign, Inc.Series B",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-14-216490.txt,726735.0,20140529.0,ANCHOR SERIES TRUST,20140331,NaN,NaN,NaN,yes,NaN,NaN
8,2594,34065,"Docusign, Inc.Series B-1",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-14-216490.txt,726735.0,20140529.0,ANCHOR SERIES TRUST,20140331,NaN,NaN,NaN,yes,NaN,NaN
9,6224,81736,"Docusign, Inc.Series D",(1)(3)(4)†,Capital Appreciation Portfolio,Capital Appreciation Portfolio,NaN,0001193125-14-216490.txt,726735.0,20140529.0,ANCHOR SERIES TRUST,20140331,NaN,NaN,NaN,yes,NaN,NaN


In [192]:
cols = []
for i in stata.columns:
    if 'Unnamed' not in i:
        cols.append(i)
        
    else:
        stata.loc[:, i] = stata.loc[:, i].astype(str)
        
stata = stata[cols]
print(stata.head(5))
stata.to_pickle('cleaned_data.pkl')

  holdings shares holdings value               holdings name         key  \
0          418804         232743  Foreign exchange contracts         (4)   
1          369192        3340855               Dropbox, Inc.  (1)(2)(3)†   
2          369192        3371831               Dropbox, Inc.  (1)(2)(3)†   
3          353044        3847591               Dropbox, Inc.  (1)(3)(4)†   
4          353044        6743564               Dropbox, Inc.  (1)(3)(4)†   

                         fund name                       identifier  \
0  Strategic Multi-Asset Portfolio  Strategic Multi-Asset Portfolio   
1   Capital Appreciation Portfolio   Capital Appreciation Portfolio   
2   Capital Appreciation Portfolio   Capital Appreciation Portfolio   
3   Capital Appreciation Portfolio   Capital Appreciation Portfolio   
4   Capital Appreciation Portfolio   Capital Appreciation Portfolio   

  value multiplier                  textfile       CIK  date_filed  \
0              NaN  0000950123-11-100758.txt  

# Filter the securities for additional screening

In [193]:
filter_list = ['class nav', 'foreign exchange',"treasury note","date counterparty","futures contracts","total credit","total options","total liability","battery park city","erie county","long island","metro. new York","new York","New York State Dormitory"",New York State Environmental","New York State Housing Finance Agency,New York State Thruway Auth","New York State Urban Dev. Corp", "Orange County NY","Port Auth. of New York & New Jersey", "Tobacco Settlement Financing Corp",\
               "Triborough Bridge & Tunnel Auth","Puerto Rico GO", "Puerto Rico Public", 'cash investments', ]
filter_keys= ['Expiration Date', 'Strike Price']

In [194]:
stata = stata[~stata['holdings name'].isna()]
stata = stata[~stata['holdings value'].isna()]
stata = stata[~stata['holdings shares'].isna()]
stata = stata[~stata['holdings name'].str.isdigit()]
stata = stata[~stata['holdings name'].astype('str').str[0].str.isdigit()]

stata = stata.loc[stata.key.notnull(), :]
stata['holdings name'] = stata['holdings name'].str.lower()
for f in filter_list:
    check = f.lower()
    stata = stata[~stata['holdings name'].str.contains(check,na=False)]
    
print(stata.head(5))

TypeError: bad operand type for unary ~: 'float'

In [183]:
#convert all data types to strings so I can export to stata file

for c in stata.columns:
    stata.loc[:,c] = stata.loc[:,c].astype(str).apply(lambda x: x.encode("ascii", "ignore").decode())
stata.to_stata('restricted_securities.dta')